In [4]:

import numpy as np

def GaussSet(Gauss_Num = 2, cuda=False):
    if Gauss_Num == 2:
        Gauss_Weight1D = [1, 1]
        Gauss_Point1D = [-1/np.sqrt(3), 1/np.sqrt(3)]
       
    elif Gauss_Num == 3:
        Gauss_Weight1D = [0.55555556, 0.88888889, 0.55555556]
        Gauss_Point1D = [-0.7745966, 0, 0.7745966]
       
        
    elif Gauss_Num == 4:
        Gauss_Weight1D = [0.3478548451374538, 0.6521451548625461, 0.6521451548625461, 0.3478548451374538]
        Gauss_Point1D = [-0.8611363115940526, -0.3399810435848563, 0.3399810435848563, 0.8611363115940526]

    elif Gauss_Num == 6: # double checked, 16 digits
        Gauss_Weight1D = [0.1713244923791704, 0.3607615730481386, 0.4679139345726910, 
                          0.4679139345726910, 0.3607615730481386, 0.1713244923791704]
        Gauss_Point1D = [-0.9324695142031521, -0.6612093864662645, -0.2386191860831969, 
                         0.2386191860831969, 0.6612093864662645, 0.9324695142031521]
    elif Gauss_Num == 8: # double checked, 20 digits
        Gauss_Weight1D=[0.10122853629037625915, 0.22238103445337447054, 0.31370664587788728733, 0.36268378337836198296,
                        0.36268378337836198296, 0.31370664587788728733, 0.22238103445337447054,0.10122853629037625915]
        Gauss_Point1D=[-0.960289856497536231684, -0.796666477413626739592,-0.525532409916328985818, -0.183434642495649804939,
                        0.183434642495649804939,  0.525532409916328985818, 0.796666477413626739592,  0.960289856497536231684]
        
    elif Gauss_Num == 10:
        Gauss_Weight1D=[0.0666713443086881, 0.1494513491505806, 0.2190863625159820, 0.2692667193099963, 0.2955242247147529,
                        0.2955242247147529, 0.2692667193099963, 0.2190863625159820, 0.1494513491505806, 0.0666713443086881]
        Gauss_Point1D=[-0.9739065285171717, -0.8650633666889845, -0.6794095682990244, -0.4333953941292472, -0.1488743389816312,  
                        0.1488743389816312,  0.4333953941292472,  0.6794095682990244,  0.8650633666889845,  0.9739065285171717]
        
    elif Gauss_Num == 20:
        Gauss_Weight1D=[0.017614007, 0.04060143, 0.062672048, 0.083276742,0.10193012, 0.118194532,0.131688638,
                        0.142096109, 0.149172986, 0.152753387,0.152753387,0.149172986, 0.142096109, 0.131688638,
                        0.118194532,0.10193012, 0.083276742,0.062672048,0.04060143,0.017614007]
            
        Gauss_Point1D=[-0.993128599, -0.963971927, -0.912234428, -0.839116972, -0.746331906, -0.636053681,
                        -0.510867002, -0.373706089, -0.227785851, -0.076526521, 0.076526521, 0.227785851,
                        0.373706089, 0.510867002, 0.636053681, 0.746331906, 0.839116972, 0.912234428, 0.963971927, 0.993128599]
    
    return Gauss_Weight1D, Gauss_Point1D
def get_quad_points(Gauss_Num, dim):

    """ Quadrature point and weight generator
    --- Inputs ---
    --- Outputs ---
    """
    Gauss_Weight1D, Gauss_Point1D = GaussSet(Gauss_Num)
    quad_points, quad_weights = [], []
    
    for ipoint, iweight in zip(Gauss_Point1D, Gauss_Weight1D):
        if dim == 1:
            quad_points.append([ipoint])
            quad_weights.append(iweight)
        else:
            for jpoint, jweight in zip(Gauss_Point1D, Gauss_Weight1D):
                if dim == 2:
                    quad_points.append([ipoint, jpoint])
                    quad_weights.append(iweight * jweight)
                else: # dim == 3
                    for kpoint, kweight in zip(Gauss_Point1D, Gauss_Weight1D):
                        quad_points.append([ipoint, jpoint, kpoint])
                        quad_weights.append(iweight * jweight * kweight)
    
    quad_points = np.array(quad_points) # (quad_degree*dim, dim)
    quad_weights = np.array(quad_weights) # (quad_degree,)
    return quad_points, quad_weights
def uniform_mesh(L, nelem_x, dim, nodes_per_elem, elem_type, non_uniform_mesh_bool=False):
    """ Mesh generator
    --- Inputs ---
    L: length of the domain
    nelem_x: number of elements in x-direction
    dim: problem dimension
    nodes_per_elem: number of nodes in one elements
    elem_type: element type
    --- Outputs ---
    XY: nodal coordinates (nnode, dim)
    Elem_nodes: elemental nodes (nelem, nodes_per_elem)
    connectivity: elemental connectivity (nelem, node_per_elem*dim)
    nnode: number of nodes
    dof_global: global degrees of freedom
    """
    
    if elem_type == 'D1LN2N': # 1D 2-node linear element
        nelem = nelem_x
        nnode = nelem+1 # number of nodes
        dof_global = nnode*dim    
        
        ## Nodes ##
        XY = np.zeros([nnode, dim], dtype=np.double)
        dx = L/nelem # increment in the x direction
    
        n = 0 # This will allow us to go through rows in NL
        for i in range(1, nelem+2):
            if i == 1 or i == nelem+1: # boundary nodes
                XY[n,0] = (i-1)*dx
            else: # inside nodes
                XY[n,0] = (i-1)*dx
                if non_uniform_mesh_bool:
                     XY[n,0] += np.random.normal(0,0.2,1)*dx# for x values
            n += 1
            
        ## elements ##
        Elem_nodes = np.zeros([nelem, nodes_per_elem], dtype=np.int32)
        for j in range(1, nelem+1):
            Elem_nodes[j-1, 0] = j-1
            Elem_nodes[j-1, 1] = j 
                   
    return XY, Elem_nodes, nelem, nnode, dof_global
def get_shape_val_functions(elem_type):
    """ Shape function generator
    """
    ############ 1D ##################
    if elem_type == 'D1LN2N': # 1D linear element
        f1 = lambda x: 1./2.*(1 - x[0])
        f2 = lambda x: 1./2.*(1 + x[0]) 
        shape_fun = [f1, f2]
    
    return shape_fun
def get_shape_vals(Gauss_Num, dim, elem_type):
    """ Meature shape function values at quadrature points
    """
    shape_val_fns = get_shape_val_functions(elem_type)
    quad_points, quad_weights = get_quad_points(Gauss_Num, dim)
    shape_vals = []
    for quad_point in quad_points:
        physical_shape_vals = []
        for shape_val_fn in shape_val_fns:
            physical_shape_val = shape_val_fn(quad_point) 
            physical_shape_vals.append(physical_shape_val)
 
        shape_vals.append(physical_shape_vals)

    shape_vals = np.array(shape_vals) # (quad_num, nodes_per_elem)
    return shape_vals



def b_fun_1D(x, L):
    # x is a scalar, a is a parameter, mostly being np.pi
    b = 1
    return b

def get_shape_grads(Gauss_Num, dim,  XY, Elem_nodes):
    quad_points, quad_weights = get_quad_points(Gauss_Num, dim)
    shape_grads =np.ones([Gauss_Num,2,1])
    shape_grads[:,0,:]=-0.5
    shape_grads[:,1,:]=0.5
    physical_coos = np.take(XY, Elem_nodes, axis=0) # (nelem, nodes_per_elem, dim)
    jacobian_dx_deta = np.sum(physical_coos[:, None, :, :, None] * shape_grads[None, :, :, None, :], axis=2, keepdims=True) # dx/deta
    # (nelem, quad_num, nodes_per_elem, dim, dim) -> (nelem, quad_num, 1, dim, dim)
    
    jacbian_det = np.squeeze(np.linalg.det(jacobian_dx_deta)) # det(J) (nelem, quad_num)
    jacobian_deta_dx = np.linalg.inv(jacobian_dx_deta) # deta/dx (nelem, quad_num, 1, dim, dim)
    shape_grads_physical = (shape_grads[None, :, :, None, :] @ jacobian_deta_dx)[:, :, :, 0, :]
    JxW = jacbian_det * quad_weights[None, :]
    return shape_grads_physical, JxW 


def get_PE(u, XY, Elem_nodes, Gauss_Num_FEM, dim, elem_type, nodes_per_elem, dof_global):
    """ Compute potential energy (PE) for HiDeNN-FEM
    """
    quad_num_FEM = Gauss_Num_FEM ** dim
    nelem = len(Elem_nodes)
    
    shape_vals = get_shape_vals(Gauss_Num_FEM, dim, elem_type) # (quad_num, nodes_per_elem)  
    shape_grads_physical, JxW = get_shape_grads(Gauss_Num_FEM, dim, elem_type, XY, Elem_nodes) # (nelem, quad_num, nodes_per_elem, dim)
    
    # Strain energy
    u_coos = np.take(u, Elem_nodes, axis=0) # (nelem, nodes_per_elem)
    Grad_uh = np.squeeze(np.sum(shape_grads_physical[:, :, :, :] * u_coos[:, None, :, None], axis=2)) # (nelem, quad_num)
    U = 0.5 * E * A * np.sum(Grad_uh**2 * JxW) 
    
    # Virtual work
    uh = np.sum(shape_vals[None, :, :] * u_coos[:, None, :], axis=2) # (nelem, quad_num)
    physical_coos = np.take(XY, Elem_nodes, axis=0) # (nelem, nodes_per_elem, dim)
    XYs = np.sum(shape_vals[None, :, :, None] * physical_coos[:, None, :, :], axis=2) # (nelem, quad_num, dim)
    body_force = vv_b_fun(XYs, L).reshape(nelem, quad_num_FEM) # (nelem, quad_num)
    W = np.sum(A * body_force * uh * JxW)
    
    return U - W 

def get_residual(sol, A_sp, b, disp_BC_idx):
    res = b - A_sp @ sol
    res = res.at[disp_BC_idx].set(0) # disp. BC
    return res


def get_Ap(p, A_sp, disp_BC_idx):
    Ap = A_sp @ p
    Ap = Ap.at[disp_BC_idx].set(0) # disp. BC
    return Ap

def CG_solver_PC(M_inv, get_residual, get_Ap, sol, A_sp, b, disp_BC_idx, dof_global, tol):
    # M_inv: inverse of the preconditioner. For my problem, it should be a vector of the diagonal.
    
    start_time = time.time()

    r_old = get_residual(sol, A_sp, b, disp_BC_idx)
    z_old = M_inv * r_old
    p = z_old
    
    for step in range(dof_global*1000000):
        # print(p)
        Ap = get_Ap(p, A_sp, disp_BC_idx)
        alpha = np.dot(r_old, z_old) / np.dot(p, Ap)
        sol += alpha * p
        r_new = r_old - alpha * Ap
        rsnew = np.dot(r_new,r_new)
        if step%1000 == 0:
            print(f"step = {step}, res l_2 = {rsnew**0.5}") 
        
        if rsnew**0.5 < tol:
            
            break
        z_new = M_inv * r_new
        beta = np.dot(r_new, z_new) / np.dot(r_old, z_old)
        p = z_new + beta * p
        
        r_old = r_new
        z_old = z_new
    print(f"CG solver took {time.time() - start_time:.4f} seconds")
    return sol

def Solve(A_sp_scipy, b, dof_global, nodes_per_elem, disp_BC_idx, disp_BC):
    """ Pre-conditioned conjugate-gradient iterative solver implemented on GPU
    """

    tol = 1e-15
    sol = np.zeros(dof_global, dtype=np.double)              # (dof,)
    sol = sol.at[disp_BC_idx].set(disp_BC) # set displacement BC
    
    # preconditioned CG solver
    M = np.array(A_sp_scipy.diagonal())
    M_inv = 1/M
    
    A_sp = BCOO.from_scipy_sparse(A_sp_scipy)
    b = np.array(b)
    sol = CG_solver_PC(M_inv, get_residual, get_Ap, sol, A_sp, b, disp_BC_idx, dof_global, tol)
       
    return sol
L=10
nelem_x=10
dim=1
nodes_per_elem=2
E=1
A=1
elem_type='D1LN2N'
XY, Elem_nodes, nelem, nnode, dof_global=uniform_mesh(L,nelem_x, dim, nodes_per_elem, elem_type,True)
Gauss_Num=6

shape_grads_physical, JxW =get_shape_grads(Gauss_Num, dim,  XY, Elem_nodes)



array([[0.11341678, 0.23882409, 0.30975893, 0.30975893, 0.23882409,
        0.11341678],
       [0.06072514, 0.1278702 , 0.16584984, 0.16584984, 0.1278702 ,
        0.06072514],
       [0.05886073, 0.12394426, 0.16075783, 0.16075783, 0.12394426,
        0.05886073],
       [0.09500283, 0.20004944, 0.25946755, 0.25946755, 0.20004944,
        0.09500283],
       [0.10357163, 0.21809296, 0.2828703 , 0.2828703 , 0.21809296,
        0.10357163],
       [0.02869725, 0.06042839, 0.07837666, 0.07837666, 0.06042839,
        0.02869725],
       [0.13671532, 0.28788432, 0.37339089, 0.37339089, 0.28788432,
        0.13671532],
       [0.08947765, 0.18841496, 0.24437743, 0.24437743, 0.18841496,
        0.08947765],
       [0.10062767, 0.21189379, 0.27482987, 0.27482987, 0.21189379,
        0.10062767],
       [0.06952747, 0.14640545, 0.18989038, 0.18989038, 0.14640545,
        0.06952747]])